In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

Links to the pages for scrapping :
    
    https://www.glassdoor.co.in/Job/bengaluru-data-scientist-jobs-SRCH_IL.0,9_IC2940587_KE10,24.htm
    https://www.glassdoor.co.in/Job/kolkata-data-scientist-jobs-SRCH_IL.0,7_IC2901633_KE8,22.htm
    https://www.glassdoor.co.in/Job/chennai-data-scientist-jobs-SRCH_IL.0,7_IC2833209_KE8,22.htm
    https://www.glassdoor.co.in/Job/pune-data-scientist-jobs-SRCH_IL.0,4_IC2856202_KE5,19.htm
    https://www.glassdoor.co.in/Job/mumbai-data-scientist-jobs-SRCH_IL.0,6_IC2851180_KE7,21.htm
    https://www.glassdoor.co.in/Job/hyderabad-data-scientist-jobs-SRCH_IL.0,9_IC2865319_KE10,24.htm
    https://www.glassdoor.co.in/Job/new-delhi-data-scientist-jobs-SRCH_IL.0,9_IC2891681_KE10,24.htm
    
    https://www.glassdoor.co.in/Job/india-data-scientist-jobs-SRCH_IL.0,5_IN115_KE6,20.htm
    


This code doesnt load several pages of the webbapplication, we need to modify this to do that so that we can scrape all the jobs in India

In [42]:
jobs = []
salary = {'Company Name':[],'Job Title':[],'Salary':[]}

In [43]:
def get_jobs(keyword, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()    
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="D:/2) Coding/Jupyter Notebook/DS_salary_proj/Scrapper and data/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
    keyword = '+'.join(keyword.split())
#     url = 'https://www.glassdoor.co.in/Job/jobs.htm?sc.keyword='+keyword   
    url = 'https://www.glassdoor.co.in/Job/data-scientist-jobs-SRCH_KE0,14.htm'
    driver.get(url)
    time.sleep(3) #adding some time for the webpage to load
        
    #searching for the total number of jobs in the link
    num_jobs = int(''.join(driver.find_element_by_class_name("jobsCount").text.split()[0].split(','))) #modifiing to scrape all the jobs instead of a single job 
    print(num_jobs)
    
    #Going through each job in this page
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.
    
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  
            
            print("Progress: {}".format("" + str(len(jobs)+1) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            #removing job alert option 
            try:
                driver.find_element_by_xpath('.//span[@class="SVGInline modal_closeIcon"]').click()
                print("job alert found and removed")
            except:
                pass
                                                   
            
                job_button.click()
                time.sleep(.5)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(3)
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div> 
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1
                
            #used for debugging
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                
            jobs.append({"Job Title" : job_title,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs
            
            #-----------------------Added to extract salary from Indian company--------------------------#
            salary_company=[]
            jobtitle_company=[]
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="salary"]').click()
                
                try:
                    salaries = driver.find_elements_by_xpath('.//div[@class="hideHH alignRt avgSalaryCol"]//div[@class="strong"]')
                    salary_company = [a.text for a in salaries]  
                    jobtitles = driver.find_elements_by_xpath('.//div[@class="noPad expandHH"]//div[@class="jobTitle strong"]')
                    jobtitle_company = [a.text for a in jobtitles]
                except NoSuchElementException:
                    salary_company.append(-1)
                    jobtitle_company.append(-1)
                    
            except NoSuchElementException:  #Rarely, some job postings do not have the "salary" tab.
                salary_company.append(-1)
                jobtitle_company.append(-1)
            
            for i in range(len(jobtitle_company)):
                salary['Company Name'].append(company_name)
                salary['Job Title'].append(jobtitle_company[i])
                salary['Salary'].append(salary_company[i])
            
            
            # for debugging
            if verbose:
                print('list of the job titles: ',jobtitle_company)
                print('list of salaries: ',salary_company)
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        
            #----------------------------------------------------------------------------------------------#                

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
            print('next page')
            #we need to wait for JS to load
            time.sleep(2) 
        except NoSuchElementException:
            if num_jobs==len(jobs):
                print("Scraping over after reaching target number of jobs.\nNeeded : {}\ngot : {}.".format(num_jobs, len(jobs)))
            else:
                print("Scraping terminated before reaching target number of jobs.\nNeeded : {}\ngot : {}.".format(num_jobs, len(jobs)))
            break

#This line converts the dictionary object into a pandas DataFrame.

In [44]:
#This line will open a new chrome window and start the scraping.
get_jobs("Data Scientist", False)

1552
Progress: 1/1552
Progress: 2/1552
Progress: 3/1552
Progress: 4/1552
Progress: 5/1552
Progress: 6/1552
Progress: 7/1552
Progress: 8/1552
Progress: 9/1552
Progress: 10/1552
Progress: 11/1552
Progress: 12/1552
Progress: 13/1552
Progress: 14/1552
Progress: 15/1552
Progress: 16/1552
Progress: 17/1552
Progress: 18/1552
Progress: 19/1552
Progress: 20/1552
Progress: 21/1552
Progress: 22/1552
Progress: 23/1552
Progress: 24/1552
Progress: 25/1552
Progress: 26/1552
Progress: 27/1552
Progress: 28/1552
Progress: 29/1552
Progress: 30/1552
next page
Progress: 31/1552
job alert found and removed
Progress: 32/1552
Progress: 33/1552
Progress: 34/1552
Progress: 35/1552
Progress: 36/1552
Progress: 37/1552
Progress: 38/1552
Progress: 39/1552
Progress: 40/1552
Progress: 41/1552
Progress: 42/1552
Progress: 43/1552
Progress: 44/1552
Progress: 45/1552
Progress: 46/1552
Progress: 47/1552
Progress: 48/1552
Progress: 49/1552
Progress: 50/1552
Progress: 51/1552
Progress: 52/1552
Progress: 53/1552
Progress: 54

In [45]:
df_jobs = pd.DataFrame(jobs) 
df_jobs

,Job Title,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,Job Description – Data Scientist\n\nLocation: ...,3.5,Buckman\n3.5,Chennai,"Memphis, TN",1001 to 5000 employees,1945,Company - Private,Chemical Manufacturing,Manufacturing,₹50 to ₹100 billion (INR),-1
1,Data Scientist,Introduction\n• The role will involve running ...,3.9,IBM\n3.9,Gurgaon,"Armonk, NY",10000+ employees,1911,Company - Public,IT Services,Information Technology,₹500+ billion (INR),"Amazon, Accenture, Microsoft"
2,Data Scientist,Driving Infinite Possibilities Within A Divers...,3.8,Honeywell\n3.8,Bengaluru,"Charlotte, NC",10000+ employees,1885,Company - Public,Computer Hardware & Software,Information Technology,₹500+ billion (INR),"GE, Johnson Controls, United Technologies"
3,Data Scientist,"At Amazon, we strive to be most customer-centr...",4.2,Amazon\n4.2,Hyderabad,"Seattle, WA",10000+ employees,1994,Company - Public,Internet,Information Technology,₹500+ billion (INR),"Google, Microsoft, Walmart"
4,Data Scientist,Provide analytical insights into emerging prob...,3.6,PayPal\n3.6,Bengaluru,"San Jose, CA",10000+ employees,1998,Company - Public,Internet,Information Technology,₹500+ billion (INR),"Square, Amazon, Apple"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,Data Analyst,"Roles and Responsibilities:\nExtracting, trans...",2.6,Exela Technologies\n2.6,Pune,"Irving, TX",10000+ employees,2017,Company - Public,IT Services,Information Technology,₹100 to ₹500 billion (INR),"SourceHOV, BancTec, Transcentra"
1548,Data Modeler,• Minimum 5-8 years of working experience as D...,3.5,LTI\n3.5,Mumbai,"Mumbai, India",10000+ employees,1997,Company - Public,IT Services,Information Technology,Unknown / Non-Applicable,"Infosys, Accenture, Capgemini"
1549,Machine Learning Engineer,Looking for Data Scientists with Python/R Prog...,-1,Tekolutions.ai,India,"Mumbai, India",Unknown,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1
1550,Data Science Internship,About the company:\nSkillBit is a leading staf...,-1,SkillBit,Pune,-1,-1,-1,-1,-1,-1,-1,-1


In [46]:
df_salary = pd.DataFrame(salary) 
df_salary

,Company Name,Job Title,Salary
0,Buckman\n3.5,Intern,₹26K - ₹29K
1,Buckman\n3.5,IOT Engineer,₹766K - ₹836K
2,IBM\n3.9,Data Scientist,"₹ 12,49,740"
3,IBM\n3.9,Data Scientist,"₹ 41,484/mo"
4,IBM\n3.9,Senior Data Scientist,"₹ 30,53,418"
...,...,...,...
5363,LTI\n3.5,Project Lead,"₹ 12,91,294"
5364,LTI\n3.5,Graduate Engineer Trainee,"₹ 3,63,502"
5365,Tekolutions.ai,-1,-1
5366,SkillBit,-1,-1


In [47]:
df_jobs.to_csv("Jobs_dst.csv")

In [48]:
df_salary.to_csv("salary_dst.csv")